## How-to guide for Customer Churn use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build a customer churn prediction model using the Abacus.AI Python Client Library.

We'll be using the [Telco Customer Churn Dataset](https://s3.amazonaws.com//realityengines.exampledatasets/customer_churn/telco.csv), which contains information about multiple users, their attributes, and whether or not they churned.

1. Install the Abacus.AI library

In [1]:
!pip install abacusai

We'll also import pandas and pprint tools for visualization in this notebook.

In [2]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [3]:
#@title Abacus.AI API Key
api_key = '2fdecde877dc45fab937eff82b70eff0'  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [4]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have ML features and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model catered specifically for your data.

We'll call the `list_use_cases` method to retrieve a list of the Use Cases currently available on the Abacus.AI platform.

In [5]:
client.list_use_cases()

[UseCase(use_case='UCPLUGANDPLAY',
   pretty_name='Plug & Play Your Tensorflow Model',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='EMBEDDINGS_ONLY',
   pretty_name='Vector Matching Engine',
   description='Upload embeddings and leverage our similarity search infrastructure.. Scale to high traffic, update your index in near realtime'),
 UseCase(use_case='MODEL_WITH_EMBEDDINGS',
   pretty_name='Tensorflow Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='TORCH_MODEL_WITH_EMBEDDINGS',
   pretty_name='PyTorch Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. H

In this notebook, we're going to create a customer churn prediction model using the Telco Customer Churn dataset. The 'CUSTOMER_CHURN' use case is best tailored for this situation. For the purpose of taking an example, we will be using the [Telco Customer Churn Dataset](https://s3.amazonaws.com//realityengines.exampledatasets/customer_churn/telco.csv) that has user information, attributes, and whether or not they churned.

In [6]:
#@title Abacus.AI Use Case
use_case = 'CUSTOMER_CHURN'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what features are required for this use_case and what features are recommended.

In [7]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

{ 'allowed_feature_mappings': { 'CHURNED_YN': { 'allowed_feature_types': [ 'CATEGORICAL'],
                                                'description': 'Specifies '
                                                               'whether a user '
                                                               'has churned '
                                                               'out or not.  '
                                                               'You need some '
                                                               'examples of '
                                                               'users that '
                                                               'have churned '
                                                               'out, for our '
                                                               'AI engine to '
                                                               'create a '
                                                

Finally, let's create the project.

In [8]:
churn_project = client.create_project(name='Customer Churn Prediction', use_case=use_case)
churn_project.to_dict()

{'project_id': '1401660f7a',
 'name': 'Customer Churn Prediction',
 'use_case': 'CUSTOMER_CHURN',
 'created_at': '2021-11-23T18:45:32+00:00',
 'feature_groups_enabled': True}

**Note: When feature_groups_enabled is True then the use case supports feature groups (collection of ML features). Feature groups are created at the organization level and can be tied to a project to further use it for training ML models**

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3`, `Google Cloud Storage`, and other cloud storage buckets, you can also connect your dataset connector and pull your data from them (bigquery, snowflake, etc.). Otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using one dataset for this notebook. We'll tell Abacus.AI how the dataset should be used when creating it by tagging the dataset with a special Abacus.AI **Dataset Type**.
- [Telco Customer Churn Dataset](https://s3.amazonaws.com//realityengines.exampledatasets/customer_churn/telco.csv) (**USER_ATTRIBUTES**): 
This dataset contains information about multiple users for a specified company, along with whether or not they churned.

### Add the dataset to Abacus.AI

First we'll use Pandas to preview the file, then add it to Abacus.AI.

In [9]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/customer_churn/telco.csv')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [11]:
churn_dataset = client.create_dataset_from_file_connector(name='Telco Customer Churn',
                                     location='s3://realityengines.exampledatasets/customer_churn/telco.csv', table_name='churn_prediction', refresh_schedule = '0 12 * * *')

datasets = [churn_dataset]

## 3. Create Feature Groups and add them to your Project

Datasets are created at the organization level and can be used to create feature groups as follows:

In [13]:
feature_group = client.create_feature_group(table_name='churn_pred_fg', sql='SELECT * FROM churn_prediction')

Adding Feature Group to the project:

In [14]:
client.add_feature_group_to_project(feature_group_id=feature_group.feature_group_id,project_id = churn_project.project_id)

Setting the Feature Group type according to the use case requirements:

In [15]:
client.set_feature_group_type(feature_group_id=feature_group.feature_group_id, project_id = churn_project.project_id, feature_group_type= "USER_ATTRIBUTES")

Check current Feature Group schema:

In [16]:
client.get_feature_group_schema(feature_group_id=feature_group.feature_group_id)

[Feature(name='customerID',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_prediction',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='gender',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_prediction',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='SeniorCitizen',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_prediction',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='Partner',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_pr

#### For each **Use Case**, there are special **Column Mappings** that must be applied to a column to fulfill use case requirements. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API:

In [18]:
client.describe_use_case_requirements(use_case)[0].allowed_feature_mappings

{'CHURNED_YN': {'description': 'Specifies whether a user has churned out or not.  You need some examples of users that have churned out, for our AI engine to create a predictive model.',
  'allowed_feature_types': ['CATEGORICAL'],
  'required': True},
 'USER_ID': {'description': 'The unique identifier for the user.',
  'allowed_feature_types': ['CATEGORICAL'],
  'required': True},
 'IGNORE': {'description': 'Ignore this column in training',
  'multiple': True,
  'required': False}}

In [20]:
client.set_feature_mapping(project_id = churn_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='Churn',feature_mapping='CHURNED_YN')

[Feature(name='customerID',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_prediction',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='gender',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_prediction',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='SeniorCitizen',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_prediction',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='Partner',
   select_clause=None,
   feature_mapping=None,
   source_table='churn_pr

client.set_feature_group_column_mapping(project_id = churn_project.project_id,feature_group_id= feature_group.feature_group_id, column='customerID',column_mapping='USER_ID')

For each required Feature Group Type within the use case, you must assign the Feature group to be used for training the model:

In [ ]:
client.use_feature_group_for_training(project_id = churn_project.project_id,feature_group_id= feature_group.feature_group_id)

Now that we've our feature groups assigned, we're almost ready to train a model!

To be sure that our project is ready to go, let's call project.validate to confirm that all the project requirements have been met:

In [ ]:
churn_project.validate()

## 4. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [ ]:
churn_project.get_training_config_options()

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [ ]:
churn_model = churn_project.train_model(training_config={})
churn_model

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated:

In [ ]:
churn_model.wait_for_evaluation()

**Note that model training might take some minutes to some hours depending upon the size of datasets, complexity of the models being trained and a variety of other factors**

## **Checkpoint** [Optional]
As model training can take an hours to complete, your page could time out or you might end up hitting the refresh button, this section helps you restore your progress:

In [ ]:
!pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = '2fdecde877dc45fab937eff82b70eff0'  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
churn_project = next(project for project in client.list_projects() if project.name == 'Customer Churn Prediction')
churn_model = churn_project.list_models()[-1]
churn_model.wait_for_evaluation()

## 5. Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics:

In [ ]:
pp.pprint(churn_model.get_metrics().to_dict())

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/CUSTOMER_CHURN/training) page.

## 6. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
churn_deployment = client.create_deployment(name='Customer Churn Deployment', description='Customer Churn Prediction Model Deployment', model_id=churn_model.model_id)
churn_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = churn_project.create_deployment_token().deployment_token
deployment_token

## 7. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `predict_churn` API call below.

This command will return the probability of a user with specified attributes churning. The prediction would be performed based on the specified dataset, which, in this case, contains information about the user, their attributes, and whether or not they churned.

In [ ]:
ApiClient().predict_churn(deployment_token=deployment_token, 
               deployment_id=churn_deployment.deployment_id, 
               query_data={"MonthlyCharges":69.7,"TotalCharges":560.85,"gender":"Male","SeniorCitizen":"1","Partner":"No","Dependents":"No","tenure":"8","PhoneService":"Yes","MultipleLines":"No","InternetService":"Fiber optic","OnlineSecurity":"No","OnlineBackup":"No","DeviceProtection":"No","TechSupport":"No","StreamingTV":"No","StreamingMovies":"No","Contract":"Month-to-month","PaperlessBilling":"Yes","PaymentMethod":"Electronic check"})